# **Chapter 2: Being Clear and Direct**

---
**Lesson:**

Llama2 responds best to clear and direct instructions. 

> Note: the *Golden Rule of Clear Prompting: show your prompt to a friend* and ask them if they could follow the intructions themselves and produce the exact result you want. If they are confused, Llama2will be confused as well. 

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 1024,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

**Example 2.1 - Haiku**

In [ ]:
# create the prompt
prompt_data = """
<s>[INST] 

<<SYS>>
<</SYS>>

Write a haiku about robots.[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 2.2 - The Best**	

In [ ]:
prompt_data = """
<s>[INST] 

<<SYS>>
<</SYS>>

Who is the best basketball player of all time?.[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 2.3 - The Best of the Best**	

In [ ]:
prompt_data = """
[INST]
<s>[INST] 

<<SYS>>
<</SYS>>

Who is the best basketball player of all time?. Make an argument for one player only.[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 2.4 - One Player Only**

Modify the basketball player prompt so that Claude doesn't equivocate at all and responds with ONLY the name of one specific player, with no other words or punctuation.

In [ ]:
prompt_data = """
<s>[INST] 

<<SYS>>
<</SYS>>

Who is the overall best basketball player of all time? Make the case for one player. Do not state your reasoning, only respond with the name of the player.[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# Chapter 2 - END.